In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd "/content/gdrive/MyDrive/Colab Notebooks/IT401_Project/"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/.shortcut-targets-by-id/1CEqRjHhKbolnL20_xSLAL03uf5-jCk7y/IT401_Project


In [ ]:
import shutil
import tqdm
import numpy as np
import cv2
import os
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model

In [ ]:
train_path = "Data/training_data"
test_path = "Data/testing_data"
batch_size = 320
learning_rate = 0.0007
epochs = 150
latent_dim = 512
num_encoder_tokens = 4096
num_decoder_tokens = 1500
time_steps_encoder = 80
max_probability = -1
save_model_path = 'model_final'
validation_split = 0.15
max_length = 10
search_type = 'greedy'

In [ ]:
# #extract features for test files

# def video_to_frames(video):
#     path = os.path.join(test_path, 'temporary_images')
#     if os.path.exists(path):
#         shutil.rmtree(path)
#     os.makedirs(path)
#     video_path = os.path.join(test_path, 'video', video)
#     count = 0
#     image_list = []
#     # Path to video file
#     cap = cv2.VideoCapture(video_path)
#     while cap.isOpened():
#         ret, frame = cap.read()
#         if ret is False:
#             break
#         cv2.imwrite(os.path.join(test_path, 'temporary_images', 'frame%d.jpg' % count), frame)
#         image_list.append(os.path.join(test_path, 'temporary_images', 'frame%d.jpg' % count))
#         count += 1

#     cap.release()
#     cv2.destroyAllWindows()
#     return image_list


# def model_cnn_load():
#     model = VGG16(weights="imagenet", include_top=True, input_shape=(224, 224, 3))
#     out = model.layers[-2].output
#     model_final = Model(inputs=model.input, outputs=out)
#     return model_final


# def load_image(path):
#     img = cv2.imread(path)
#     img = cv2.resize(img, (224, 224))
#     return img


# def extract_features(video, model):
#     """

#     :param video: The video whose frames are to be extracted to convert into a numpy array
#     :param model: the pretrained vgg16 model
#     :return: numpy array of size 4096x80
#     """
#     video_id = video.split(".")[0]
#     print(video_id)
#     print(f'Processing video {video}')

#     image_list = video_to_frames(video)
#     samples = np.round(np.linspace(
#         0, len(image_list) - 1, 80))
#     image_list = [image_list[int(sample)] for sample in samples]
#     images = np.zeros((len(image_list), 224, 224, 3))
#     for i in range(len(image_list)):
#         img = load_image(image_list[i])
#         images[i] = img
#     images = np.array(images)
#     fc_feats = model.predict(images, batch_size=128)
#     img_feats = np.array(fc_feats)
#     # cleanup
#     shutil.rmtree(os.path.join(test_path, 'temporary_images'))
#     return img_feats


# def extract_feats_pretrained_cnn():
#     """
#     saves the numpy features from all the videos
#     """
#     model = model_cnn_load()
#     print('Model loaded')

#     if not os.path.isdir(os.path.join(test_path, 'feat')):
#         os.mkdir(os.path.join(test_path, 'feat'))

#     video_list = os.listdir(os.path.join(test_path, 'video'))
#     for video in video_list:

#         outfile = os.path.join(test_path, 'features_dir', video.split(".")[0] + '.npy')
#         img_feats = extract_features(video, model)
#         np.save(outfile, img_feats)


# if __name__ == "__main__":
#     extract_feats_pretrained_cnn()


Model loaded
0lh_UWF9ZP4_148_155
Processing video 0lh_UWF9ZP4_148_155.avi
1/1 [==============================] - 0s 210ms/step
-wa0umYJVGg_139_157
Processing video -wa0umYJVGg_139_157.avi
1/1 [==============================] - 0s 113ms/step
-dm-ds5rRaM_44_52
Processing video -dm-ds5rRaM_44_52.avi
1/1 [==============================] - 0s 64ms/step
0lh_UWF9ZP4_79_82
Processing video 0lh_UWF9ZP4_79_82.avi
1/1 [==============================] - 0s 61ms/step
3chNlP5TeO8_0_10
Processing video 3chNlP5TeO8_0_10.avi
1/1 [==============================] - 0s 64ms/step
0lh_UWF9ZP4_174_178
Processing video 0lh_UWF9ZP4_174_178.avi
1/1 [==============================] - 0s 82ms/step
_O9kWD8nuRU_37_43
Processing video _O9kWD8nuRU_37_43.avi
1/1 [==============================] - 0s 66ms/step
4Hhdr1IPOGs_1_10
Processing video 4Hhdr1IPOGs_1_10.avi
1/1 [==============================] - 0s 64ms/step
0xx13BuvVmo_25_36
Processing video 0xx13BuvVmo_25_36.avi
1/1 [==============================] - 0s 61ms/s

In [ ]:
#extract features for training files


def video_to_frames(video):
    path = os.path.join(train_path, 'temporary_images')
    if os.path.exists(path):
        shutil.rmtree(path)
    os.makedirs(path)
    video_path = os.path.join(train_path, 'video', video)
    count = 0
    image_list = []
    # Path to video file
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if ret is False:
            break
        cv2.imwrite(os.path.join(train_path, 'temporary_images', 'frame%d.jpg' % count), frame)
        image_list.append(os.path.join(train_path, 'temporary_images', 'frame%d.jpg' % count))
        count += 1

    cap.release()
    cv2.destroyAllWindows()
    return image_list


def model_cnn_load():
    model = VGG16(weights="imagenet", include_top=True, input_shape=(224, 224, 3))
    out = model.layers[-2].output
    model_final = Model(inputs=model.input, outputs=out)
    return model_final


def load_image(path):
    img = cv2.imread(path)
    img = cv2.resize(img, (224, 224))
    return img


def extract_features(video, model):
    """

    :param video: The video whose frames are to be extracted to convert into a numpy array
    :param model: the pretrained vgg16 model
    :return: numpy array of size 4096x80
    """
    video_id = video.split(".")[0]
    print(video_id)
    print(f'Processing video {video}')
    

    image_list = video_to_frames(video)
    samples = np.round(np.linspace(
        0, len(image_list) - 1, 80))
    image_list = [image_list[int(sample)] for sample in samples]
    images = np.zeros((len(image_list), 224, 224, 3))
    for i in range(len(image_list)):
        img = load_image(image_list[i])
        images[i] = img
    images = np.array(images)
    fc_feats = model.predict(images, batch_size=128)
    img_feats = np.array(fc_feats)
    # cleanup
    shutil.rmtree(os.path.join(train_path, 'temporary_images'))
    return img_feats


def extract_feats_pretrained_cnn():
    """
    saves the numpy features from all the videos
    """
    model = model_cnn_load()
    print('Model loaded')
    counter=0

    if not os.path.isdir(os.path.join(train_path, 'feat')):
        os.mkdir(os.path.join(train_path, 'feat'))

    video_list = os.listdir(os.path.join(train_path, 'video'))
    for video in video_list:
        if(counter<1879):
          counter+=1
          continue
        outfile = os.path.join(train_path, 'features_dir', video.split(".")[0] + '.npy')
        img_feats = extract_features(video, model)
        np.save(outfile, img_feats)
        counter+=1
        print(counter)

if __name__ == "__main__":
    extract_feats_pretrained_cnn()
    


Model loaded
0lh_UWF9ZP4_38_46
Processing video 0lh_UWF9ZP4_38_46.avi
1/1 [==============================] - 0s 176ms/step
1880
3I9tSDU-OSk_17_22
Processing video 3I9tSDU-OSk_17_22.avi
1/1 [==============================] - 0s 61ms/step
1881
FwCmcZpkk-k_22_32
Processing video FwCmcZpkk-k_22_32.avi
1/1 [==============================] - 0s 65ms/step
1882
MqS82kBtUHQ_0_12
Processing video MqS82kBtUHQ_0_12.avi
1/1 [==============================] - 0s 61ms/step
1883
a8PAY4rvzRM_1_10
Processing video a8PAY4rvzRM_1_10.avi
1/1 [==============================] - 0s 60ms/step
1884
NoCgvH4xZHo_240_251
Processing video NoCgvH4xZHo_240_251.avi
1/1 [==============================] - 0s 59ms/step
1885
WTf5EgVY5uU_32_52
Processing video WTf5EgVY5uU_32_52.avi
1/1 [==============================] - 0s 60ms/step
1886
